# Pulizia releases.csv

In [1]:
import pandas as pd
import os

In [1]:
import os
import pandas as pd

# Ottiene la directory di lavoro corrente
script_dir = os.getcwd()

# Risali alla cartella "Solutions"
solutions_dir = os.path.abspath(os.path.join(script_dir, ".."))

# Imposta "Data" come directory di lavoro
data_dir = os.path.join(solutions_dir, "Data")
os.chdir(data_dir)  # Cambia la directory di lavoro

# Ora aggiorna la directory corrente dopo il cambio
script_dir = os.getcwd()

# Costruisce il percorso relativo al file CSV
file_path = os.path.join(script_dir, "main_data", "releases.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "clean_data")

# Crea la cartella clean_data se non esiste
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

# Controlla che il file esista prima di caricarlo
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Il file non esiste: {file_path}")

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)
df.head()

,id,country,date,type,rating
0,1000001,Andorra,2023-07-21,Theatrical,NaN
1,1000001,Argentina,2023-07-20,Theatrical,ATP
2,1000001,Australia,2023-07-19,Theatrical,PG
3,1000001,Australia,2023-10-01,Digital,PG
4,1000001,Austria,2023-07-20,Theatrical,NaN


In [3]:
# Rimuove duplicati
df.drop_duplicates(inplace=True)

In [4]:
# Rimuove righe con troppi valori nulli (ad esempio, più del 50% dei dati mancanti)
df.dropna(thresh=df.shape[1] * 0.5, inplace=True)

# Sostituisce i valori nulli con "Unknown"
df['country'] = df['country'].fillna("Unknown")
df['date'] = df['date'].fillna("Unknown")
df['type'] = df['type'].fillna("Unknown")
df['rating'] = df['rating'].fillna("Unknown")

In [5]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "releases_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

# Caricamento file csv pulito nel database con MongoDB


In [6]:
from pymongo import MongoClient

In [8]:
# Configura MongoDB (modifica se necessario)
MONGO_URL = "mongodb://localhost:27017/"
DB_NAME = "ium_database"
COLLECTION_NAME = "releases_data"

# Connessione a MongoDB
client = MongoClient(MONGO_URL)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

# Carica il CSV con Pandas
df = pd.read_csv(cleaned_csv_path)

# Converte il DataFrame in JSON-like per MongoDB
data = df.to_dict(orient="records")

# Usa insert_many() per inserire tutto in un'unica operazione (molto più veloce)
collection.insert_many(data, ordered=False); #il ; non fa stampare l'output su jupyter